In [5]:
import pandas as pd
import numpy as np
import covidcast
from datetime import date, datetime, timedelta
import os


In [6]:
os.chdir('/Users/hongrudu/Desktop/COVID_Vaccination_rate/data/raw_data')

In [7]:
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

In [8]:
def create_relative(df, col):
    df_ref = df[['Week_Mon', col]].groupby(['Week_Mon']).mean().reset_index()
    df_ref = df_ref.rename(columns = {col : 'ref'})
    df = df.merge(df_ref, left_on = ['Week_Mon'], right_on = ['Week_Mon'])
    df['Relative_' + col] = df[col]/df['ref']
    df = df.drop(columns = ['ref'])
    return df

In [9]:
cases = pd.read_pickle('Weekly_cases.pkl')
hospitalized = pd.read_pickle('Weekly_hospitalization.pkl')
df = hospitalized.merge(cases)
#### Smooth data
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases']] =\
    df[df['FIPS'] == fips][['Cases']].rolling(window = 3).mean()
df = df.dropna().reset_index().drop(columns = ['index'])

In [10]:
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases_lag1']] =\
    df[df['FIPS'] == fips]['Cases'].shift(2)
df = df.dropna().reset_index().drop(columns = ['index'])
df['IHR_lag1'] = df['hospitalized']/df['Cases_lag1']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['IHR_lag1']] =\
    df[df['FIPS'] == fips][['IHR_lag1']].rolling(window = 3).mean()

In [11]:
pop = pd.read_pickle('State_population.pkl')
df = pop.merge(df)

In [12]:
#### Vaccine Data
vaccine = pd.read_pickle('Weekly_vaccination.pkl')
df = df.merge(vaccine)
for i in vaccine.columns[2:]:
    df[i] = df[i]/df['Population']
for i in vaccine.columns[2:]:
    df[i] = df[i]/7

In [13]:
for fips in ['35', '42']:
    df.loc[df[df['FIPS'] == fips].index, ['People_fully_vaccinated']] =\
    df[df['FIPS'] == fips][['People_fully_vaccinated']].rolling(window = 3).mean()

In [14]:
##### Policy data
policy = pd.read_pickle('Weekly_policy.pkl')
df = df.merge(policy)

In [15]:
###### Testing data
test = pd.read_pickle('Weekly_testing.pkl')
test.Weekly_testing = test.Weekly_testing.mask(test.Weekly_testing.lt(0),np.nan)
test['Weekly_testing'] = test['Weekly_testing'].interpolate(method='linear')
df = df.merge(test)
for i in test.columns[2:]:
    df[i] = df[i]/df['Population']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Weekly_testing']] =\
    df[df['FIPS'] == fips][['Weekly_testing']].rolling(window = 3).mean()

In [16]:
#### Past infections
past_inf = pd.read_pickle('Weekly_past_infection.pkl')
df = df.merge(past_inf)
for i in past_inf.columns[2:]:
    df[i] = df[i]/df['Population']

In [17]:
# df.to_pickle('processed_data/var_before_trans.pkl')

In [18]:
##### Add lags

for col in ['People_at_least_one_dose', 'People_fully_vaccinated',
       'Total_additional_doses', 'GovernmentResponseIndex_Average',
       'Weekly_testing']:
    for fips in df['FIPS'].unique():
         df.loc[df[df['FIPS'] == fips].index, col + '_lag2'] =\
         df[df['FIPS'] == fips][col].shift(2)

for col in ['Past_16_week_infections', 'Past_12_week_infections',
       'Past_20_week_infections', 'Past_24_week_infections']:
    for fips in df['FIPS'].unique():
         df.loc[df[df['FIPS'] == fips].index, col + '_lag4'] =\
         df[df['FIPS'] == fips][col].shift(4)
            
df = df[~df['FIPS'].isin(['33', '54'])].reset_index().drop(columns = ['index'])
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['IHR_lag1']] =\
    df[df['FIPS'] == fips][['IHR_lag1']].rolling(window = 3).mean()

In [19]:
df_selected = df[['FIPS', 'Week_Mon', 'IHR_lag1', 'People_at_least_one_dose_lag2', 'People_fully_vaccinated_lag2',
       'Total_additional_doses_lag2', 'GovernmentResponseIndex_Average_lag2',
       'Weekly_testing_lag2', 'Past_16_week_infections_lag4',
       'Past_12_week_infections_lag4', 'Past_20_week_infections_lag4',
       'Past_24_week_infections_lag4']]

In [20]:
for col in df_selected.columns[2:]:
    df_selected = create_relative(df_selected, col)

In [22]:
mobility = pd.read_pickle('Weekly_activity_level.pkl')
mobility['Week_Mon'] = pd.to_datetime(mobility['Week_Mon'])
for fips in mobility['FIPS'].unique():
    mobility.iloc[mobility[mobility['FIPS'] == fips].index,2:] =\
    mobility[mobility['FIPS'] == fips].iloc[:, 2:].rolling(window = 3).mean()

In [23]:
df_selected = df_selected.merge(mobility, how = 'left')

In [24]:
for col in mobility.columns[2:]:
    for fips in df_selected['FIPS'].unique():
         df_selected.loc[df_selected[df_selected['FIPS'] == fips].index, col + '_lag2'] =\
         df_selected[df_selected['FIPS'] == fips][col].shift(2)

In [25]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_IHR_lag1', 'Relative_People_at_least_one_dose_lag2',
       'Relative_People_fully_vaccinated_lag2',
       'Relative_Total_additional_doses_lag2',
       'Relative_GovernmentResponseIndex_Average_lag2',
       'Relative_Weekly_testing_lag2', 'Relative_Past_16_week_infections_lag4',
       'Relative_Past_12_week_infections_lag4',
       'Relative_Past_20_week_infections_lag4',
       'Relative_Past_24_week_infections_lag4',
       'University_lag2', 'full_service_restaurant_lag2',
       'limited_service_restaurant_lag2', 'elementary_school_lag2',
       'other_general_store_lag2', 'gas_station_lag2', 'gym_lag2',
       'grocery_store_lag2', 'cafes_snack_bar_lag2', 'hotel_lag2',
       'religious_organization_lag2', 'parks_lag2', 'hardware_store_lag2',
       'department_store_lag2', 'child_day_care_lag2',
       'office_of_physician_lag2', 'pharmacies_lag2',
       'sporting_goods_store_lag2', 'automotive_store_lag2',
       'used_merchandise_store_lag2', 'convenience_store_lag2']]

In [26]:
for col in ['University_lag2', 'full_service_restaurant_lag2',
       'limited_service_restaurant_lag2', 'elementary_school_lag2',
       'other_general_store_lag2', 'gas_station_lag2', 'gym_lag2',
       'grocery_store_lag2', 'cafes_snack_bar_lag2', 'hotel_lag2',
       'religious_organization_lag2', 'parks_lag2', 'hardware_store_lag2',
       'department_store_lag2', 'child_day_care_lag2',
       'office_of_physician_lag2', 'pharmacies_lag2',
       'sporting_goods_store_lag2', 'automotive_store_lag2',
       'used_merchandise_store_lag2', 'convenience_store_lag2']:
    df_selected = create_relative(df_selected, col)

In [27]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_IHR_lag1',
       'Relative_People_at_least_one_dose_lag2',
       'Relative_People_fully_vaccinated_lag2',
       'Relative_Total_additional_doses_lag2',
       'Relative_GovernmentResponseIndex_Average_lag2',
       'Relative_Weekly_testing_lag2', 'Relative_Past_16_week_infections_lag4',
       'Relative_Past_12_week_infections_lag4',
       'Relative_Past_20_week_infections_lag4',
       'Relative_Past_24_week_infections_lag4',
       'Relative_University_lag2', 'Relative_full_service_restaurant_lag2',
       'Relative_limited_service_restaurant_lag2',
       'Relative_elementary_school_lag2', 'Relative_other_general_store_lag2',
       'Relative_gas_station_lag2', 'Relative_gym_lag2',
       'Relative_grocery_store_lag2', 'Relative_cafes_snack_bar_lag2',
       'Relative_hotel_lag2', 'Relative_religious_organization_lag2',
       'Relative_parks_lag2', 'Relative_hardware_store_lag2',
       'Relative_department_store_lag2', 'Relative_child_day_care_lag2',
       'Relative_office_of_physician_lag2', 'Relative_pharmacies_lag2',
       'Relative_sporting_goods_store_lag2', 'Relative_automotive_store_lag2',
       'Relative_used_merchandise_store_lag2',
       'Relative_convenience_store_lag2']]

In [28]:
genomic = pd.read_pickle('Weekly_genomic.pkl')
genomic['Week_Mon'] = pd.to_datetime(genomic['Week_Mon'])
df_selected = df_selected.merge(genomic)

In [29]:
df_selected['Dominant_variant'] = df_selected['Dominant_variant'].replace({'Alpha' : 'Others'})

In [32]:
static = pd.read_pickle('Static_data.pkl')
df_selected = df_selected.merge(static)

In [33]:
df_selected = df_selected[df_selected['Week_Mon']>='2021-03-01'].reset_index().drop(columns = ['index'])
df_selected = df_selected[df_selected['Week_Mon']<='2022-03-01'].reset_index().drop(columns = ['index'])
df_selected['medicaid_spending'] = df_selected['medicaid_spending']/1000

In [34]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_IHR_lag1',
       'Relative_People_at_least_one_dose_lag2',
       'Relative_People_fully_vaccinated_lag2',
       'Relative_Total_additional_doses_lag2',
       'Relative_GovernmentResponseIndex_Average_lag2',
       'Relative_Weekly_testing_lag2', 'Relative_Past_16_week_infections_lag4',
       'Relative_Past_12_week_infections_lag4',
       'Relative_Past_20_week_infections_lag4',
       'Relative_Past_24_week_infections_lag4', 'Relative_University_lag2',
       'Relative_full_service_restaurant_lag2','Relative_gas_station_lag2',
       'Relative_gym_lag2',
       'Relative_religious_organization_lag2',
       'Relative_office_of_physician_lag2','over_65',
       'Black', 'medicaid_spending', 'health_care_spending', 'poverty_rate',
       'social_vulnerability_index',
       'Healthcare Access and Quality Index', 'rep_voters',
       'Adults_at_high_risk', 'Dominant_variant']]

In [35]:
df_selected = df_selected.rename(columns = {'Relative_IHR_lag1' : 'RCHR'})
df_selected = df_selected.rename(columns = {'Relative_People_at_least_one_dose_lag2' : 'RPVR_lag2',
                             'Relative_People_fully_vaccinated_lag2' : 'RCPSR_lag2',
                             'Relative_Total_additional_doses_lag2' : 'RBVR_lag2',
                             'Relative_GovernmentResponseIndex_Average_lag2' : 'RGP_lag2',
                             'Relative_Weekly_testing_lag2' : 'RWTR_lag2',
                             'Relative_Past_12_week_infections_lag4' : 'RPI_12_Weeks_lag4',
                             'Relative_Past_16_week_infections_lag4' : 'RPI_16_Weeks_lag4',
                             'Relative_Past_20_week_infections_lag4' : 'RPI_20_Weeks_lag4',
                             'Relative_Past_24_week_infections_lag4' : 'RPI_24_Weeks_lag4',
                             'Relative_University_lag2' : 'RUV_lag2',
                            'Relative_full_service_restaurant_lag2' : 'RFSRV_lag2',
                             'Relative_gas_station_lag2' : 'RGSV_lag2',
                             'Relative_gym_lag2' : 'RGV_lag2', 
                             'Relative_religious_organization_lag2' : 'RROV_lag2',
                             'Relative_office_of_physician_lag2' : 'ROPV_lag2',
                             'Healthcare Access and Quality Index' : 'HAQI',
                             'social_vulnerability_index' : 'SVI'})

In [37]:
df_selected.to_csv('/Users/hongrudu/Desktop/COVID_Vaccination_rate/data/processed_data/CHR_input.csv')